## RCS Python Closures and Generators

## Closures - binding variables from outer function in the inner function
## Technically - function gets stored with its enviroment(bound variables)
### Can also think of preserving certain state

In [ ]:
# remember this function?
def add_factory(x):
    def add(y):
        return y + x 
    return add # upon return free variable x gets bound in the add function


In [ ]:
add5 = add_factory(5)
# 5 is bound inside add5 now,
add5(10)

In [ ]:
type(add5.__closure__)

In [ ]:
[x for x in add5.__closure__]

In [ ]:
len(add5.__closure__)

In [ ]:
int(add5.__closure__[0])

In [ ]:
dir(add5.__closure__[0])

In [ ]:
add5.__closure__[0].cell_contents

In [ ]:
## Voila!! We get what we expected to get!

In [ ]:
## Remember __closure__ is a tuple so we do not get to mutate this!

In [ ]:
So how about more values stored?

In [ ]:
def add2_fact(x, y):
    return lambda z: z+x+y

In [ ]:
a10n20 = add2_fact(10,20)

In [ ]:
a10n20(40)

In [ ]:
len(a10n20.__closure__)

In [ ]:
[x.cell_contents for x in a10n20.__closure__]

In [ ]:
One last closure example:

In [ ]:
def outer(x):
    a = 20
    def inner(y):
        print(f'x:{x}')
        print(f'a:{a}')
        print(f'y:{y}')
        ## x += 15 # We can't change the argument that was bound from outside argument
        ## a += 15 # We can't change the a that was bound from outside function
        return a+x+y
    return inner

In [ ]:
axy = outer(10)

In [ ]:
axy(5)

In [ ]:
axy(5)

In [ ]:
[x.cell_contents for x in axy.__closure__]

## What if we want rebind(assign new values) to variables coming from outer scope?
### In languages like Javascript you can do it, so Python should be able to, right?
### Solution: Python3 nonlocal modifier inside inner function 

In [ ]:
# https://docs.python.org/3/reference/simple_stmts.html#the-nonlocal-statement
# 7.13. The nonlocal statement

# The nonlocal statement causes the listed identifiers to refer to previously bound variables in the nearest enclosing scope excluding globals. 
# This is important because the default behavior for binding is to search the local namespace first. The statement allows encapsulated code to rebind variables outside of the local scope besides the global (module) scope.

# Names listed in a nonlocal statement, unlike those listed in a global statement, must refer to pre-existing bindings in an enclosing scope (the scope in which a new binding should be created cannot be determined unambiguously).

# Names listed in a nonlocal statement must not collide with pre-existing bindings in the local scope.

In [ ]:
def makeCounter():
    count = 5
    def f():
        nonlocal count
        count +=1
        def h():
            nonlocal count
            count +=2
            return count
        return h()
    return f


In [ ]:
a = makeCounter()

In [ ]:
a()

In [ ]:
dir(a)

In [ ]:
a()

In [ ]:
print(a(),a(),a())

In [ ]:
[a() for x in range(10)]

In [ ]:
dir(a)

In [ ]:
def makeAdjCounter(x):
    count = 0
    def f():
        nonlocal count  # without nonlocal we could reference count but couldn't modify it!
        count += x
        return count
    return f

In [ ]:
b = makeAdjCounter(2)
c = makeAdjCounter(3)

In [ ]:
print(b(),b(),b(), c(), c(), c())

In [ ]:
print(c(),c(),c(),c())

In [ ]:
[x.cell_contents for x in c.__closure__]

In [ ]:
# Result count is hidden from us, but by calling function we can modify its value.


In [ ]:
## Another older way was to create some structure(List, Class, Dictionary) inside outer function whose members could be modified by innner


In [ ]:
def makeAdjList():
    holder=[1,0,0,3] # old method not recommended anymore!
    def f():
        holder[0] +=1
        print(holder)
        return holder[0]
    return f

In [ ]:
d = makeAdjList()


In [ ]:
print(d(),d(),d())

### Most Python answer is to use generators for persisting some sort of iterable state

## What the heck is a Generator ?

###  A Python generator is a function which returns a generator iterator (just an object we can iterate over) by calling yield

* KEY POINT: generator functions use **yield** instead of **return**
* in Python 3 we use next(generatorName) to obtain next value

In [1]:
def makeNextGen(current):
    while True: ##This means our generator will never run out of values...
        current += 1
        yield current
    

In [2]:
numGen = makeNextGen(30)

In [57]:
mybyte = b'\x31\x32\x13\x07'

In [59]:
print(mybyte.decode('ascii'))

12


In [48]:
len(mybyte)

2

In [49]:
int.from_bytes(mybyte, byteorder='little')

4097

In [50]:
int.from_bytes(mybyte, byteorder='big')

272

In [26]:
type(mybyte)

bytes

In [27]:
len(mybyte)

9

In [24]:
print(mybyte)

b'3daa2'


In [9]:
type(makeNextGen)

function

In [4]:
dir(makeNextGen)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [8]:
type(range)

type

In [5]:
for i in range(20):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [ ]:
for i in range(15):
    print(next(numGen))  # This is for Python 3.x ,  in Python 2.x it was numGen.next()

In [ ]:
## DO not do this!!
#for el in numGen:
#    print(el)

In [ ]:
## We can do even better and make an adjustable increment

In [61]:
def makeNextGenInc(current, inc):
    while True: 
        current += inc
        yield current


In [ ]:
numGen = makeNextGenInc(20,5)

In [ ]:
next(numGen)

In [ ]:
def smallYield():
    yield 1
    yield 2
    yield 99
    yield 5


In [ ]:
smallGen = smallYield()

In [ ]:
next(smallGen)

In [ ]:
list(numGen)

In [ ]:
list(smallGen)

In [63]:
numGen10 = makeNextGenInc(200, 10)

In [64]:
[next(numGen10) for x in range(15)]

[210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350]

In [ ]:
## Now the above is Pythonic approach to the problem!

In [ ]:
### Then there is a generator expression 
## The whole point is have a lazy evaluation (ie no need to have everything at once in memory)


In [71]:
gen = (i+10 for i in range(10))


In [68]:
for g in gen:
    print(g)

In [67]:
list(gen)

[]

In [ ]:
## list(i+10 for i in range(10)) == [i+10 for i in range(10)]

In [69]:
type(gen)

generator

In [72]:
list(gen)

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [81]:
gen = (i+10 for i in range(10))

In [74]:
for g in gen:
    print(g)

10
11
12
13
14
15
16
17
18
19


In [75]:
for g in gen:
    print(g)

In [ ]:
# You see what is going on?!

In [79]:
gen_exp = (x ** 2 for x in range(10) if x % 2 == 0)
type(gen_exp)

generator

In [77]:
for x in gen_exp:
    print(x)

0
4
16
36
64


In [82]:
glist = list(gen)
glist

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [84]:
gen = (i+10 for i in range(10))
[next(gen) for x in range(5)]

[10, 11, 12, 13, 14]

In [85]:
yes_expr = ('yes' for _ in range(10))

In [87]:
def my_yes_gen():
    for _ in range(10):
        yield('yes')

In [ ]:
#infinite generator
def my_yes_gen():
    while True:
        yield('yes')

In [88]:
myg = my_yes_gen()

In [89]:
list(myg)

['yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes']

In [86]:
list(yes_expr)

['yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes']

In [ ]:
## Challenge how to make an infinite generator with a generator expression?

In [91]:
import itertools

In [92]:
genX = (i*5 for i in itertools.count(start=0, step=1))


In [95]:
[next(genX) for x in range(10)]

[225, 230, 235, 240, 245, 250, 255, 260, 265, 270]

In [93]:
[next(genX) for x in range(35)]

[0,
 5,
 10,
 15,
 20,
 25,
 30,
 35,
 40,
 45,
 50,
 55,
 60,
 65,
 70,
 75,
 80,
 85,
 90,
 95,
 100,
 105,
 110,
 115,
 120,
 125,
 130,
 135,
 140,
 145,
 150,
 155,
 160,
 165,
 170]

In [102]:
gendice = (random.randrange(1,7) for _ in itertools.count(start=0, step=1))
[next(gendice) for x in range(20)]

[6, 3, 4, 1, 3, 4, 2, 4, 2, 2, 2, 3, 1, 6, 4, 5, 6, 6, 5, 5]

In [96]:
import random
genY = (i*10+random.randrange(10) for i in itertools.count(start=0, step=1))

In [103]:
[next(genY) for x in range(10)]

[206, 216, 229, 237, 242, 253, 268, 271, 283, 299]

In [ ]:
## Be very careful with infinite generators, calling list on infinite generator not recommended!

In [ ]:
## Of course we should generally have a pretty good idea of maximum number of generations needed

### Difference between Python's Generators and Iterators
* iterators is more general, covers all generators

From Official Docs: Python’s generators provide a convenient way to implement the iterator protocol. If a container object’s __iter__() method is implemented as a generator, it will automatically return an iterator object (technically, a generator object) supplying the __iter__() and next()

## A Generator is an Iterator
### Specifically, generator is a subtype of iterator.

Conceptually:
Iterators are about various ways to loop over data, generators generate the data on the fly

In [104]:
import itertools

In [105]:
dir(itertools)

['__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_grouper',
 '_tee',
 '_tee_dataobject',
 'accumulate',
 'chain',
 'combinations',
 'combinations_with_replacement',
 'compress',
 'count',
 'cycle',
 'dropwhile',
 'filterfalse',
 'groupby',
 'islice',
 'permutations',
 'product',
 'repeat',
 'starmap',
 'takewhile',
 'tee',
 'zip_longest']

In [106]:
help(itertools.product)

Help on class product in module itertools:

class product(builtins.object)
 |  product(*iterables, repeat=1) --> product object
 |  
 |  Cartesian product of input iterables.  Equivalent to nested for-loops.
 |  
 |  For example, product(A, B) returns the same as:  ((x,y) for x in A for y in B).
 |  The leftmost iterators are in the outermost for-loop, so the output tuples
 |  cycle in a manner similar to an odometer (with the rightmost element changing
 |  on every iteration).
 |  
 |  To compute the product of an iterable with itself, specify the number
 |  of repetitions with the optional repeat keyword argument. For example,
 |  product(A, repeat=4) means the same as product(A, A, A, A).
 |  
 |  product('ab', range(3)) --> ('a',0) ('a',1) ('a',2) ('b',0) ('b',1) ('b',2)
 |  product((0,1), (0,1), (0,1)) --> (0,0,0) (0,0,1) (0,1,0) (0,1,1) (1,0,0) ...
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /

In [109]:
list(itertools.product(range(10),list('ABCDE')))

[(0, 'A'),
 (0, 'B'),
 (0, 'C'),
 (0, 'D'),
 (0, 'E'),
 (1, 'A'),
 (1, 'B'),
 (1, 'C'),
 (1, 'D'),
 (1, 'E'),
 (2, 'A'),
 (2, 'B'),
 (2, 'C'),
 (2, 'D'),
 (2, 'E'),
 (3, 'A'),
 (3, 'B'),
 (3, 'C'),
 (3, 'D'),
 (3, 'E'),
 (4, 'A'),
 (4, 'B'),
 (4, 'C'),
 (4, 'D'),
 (4, 'E'),
 (5, 'A'),
 (5, 'B'),
 (5, 'C'),
 (5, 'D'),
 (5, 'E'),
 (6, 'A'),
 (6, 'B'),
 (6, 'C'),
 (6, 'D'),
 (6, 'E'),
 (7, 'A'),
 (7, 'B'),
 (7, 'C'),
 (7, 'D'),
 (7, 'E'),
 (8, 'A'),
 (8, 'B'),
 (8, 'C'),
 (8, 'D'),
 (8, 'E'),
 (9, 'A'),
 (9, 'B'),
 (9, 'C'),
 (9, 'D'),
 (9, 'E')]

## Homework
### Write a generator to yield cubes (forever!)
### Write a generator to yield Fibonacci numbers(first 1000)

* Generator Functions ok to use here

In [ ]:
def fib():
    a, b = 0, 1
    while True:
        a, b = b, a+b
        yield b

In [ ]:
def fib1000():
    a, b = 0, 1
    for x in range(1000):
        a, b = b, a+b
        yield b

In [ ]:
f1k = fib1000()

In [ ]:
[next(f1k) for _ in range(10)]

In [ ]:
f = fib()


In [ ]:
[next(f) for _ in range(10)]

In [ ]:
def cubes(current):
    while True:
        #print(current**3)
        current+=1
        cube = current**3
        yield cube

In [ ]:
g3 = cubes(1)

In [ ]:
next(g3)

In [ ]:

cubesforever = (x**3 for x in itertools.count(start=0, step=1))

In [ ]:
c30 = [next(cubesforever) for _ in range(30)]

In [ ]:
c30


In [ ]:
# Hint use yield

In [ ]:
## Extra Credit! write generator expression for first 500 cubes that are made from even numbers

In [ ]:
g500 = (x**3 for x in range(1,501) if x % 2 == 0)


In [ ]:
a10 = [next(g500) for x in range(10)]


In [ ]:
a10

In [ ]:
a = list(g500)

In [ ]:
a[:10]

In [ ]:
next(g500)

In [ ]:
f10 = list(g500[:10])